# IMMC 2020
### Importing Modules and Data

In [1]:
import pandas as pd
import copy
import math, statistics
import matplotlib.pyplot as plt
from pprint import pprint

median_income = 9733/365
loss_aversion_coefficient = 2

pdt_csv_data = pd.read_csv("StoreData_IMMC_CSV.csv")

In [2]:
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

## Determining the Popularity of Product
### Impact of Discount on Popularity

In [3]:
# Traditional Econs Approach
def q1_over_q0(p0, p1, percentage_usage):
    x = (p0 - p1)*(percentage_usage/median_income) 
    return (math.exp(x))

# Behavioural Econs Approach
def prospect_utility(x):
    if x > 0:
        return(math.log(x+1))
    else:
        return(-loss_aversion_coefficient * math.log(-x + 1))

def increase_utility(p0, p1):
    return(prospect_utility(p0 - p1))

# Helper Variables
max_increase_utility = increase_utility(3329.99, 2199.99)
min_increase_utility = 0
max_q1_over_q0 = q1_over_q0(3329.99, 2199.99, 1)
min_q1_over_qo = 1

# Combined Effect [Between 0 and 1]
def popularity_due_to_discount(p0, p1, percentage_usage):
    traditional_econs_adjusted = (q1_over_q0(p0, p1, percentage_usage)-min_q1_over_qo)/max_q1_over_q0
    behavioural_econs_adjusted = (increase_utility(p0, p1)-min_increase_utility)/max_increase_utility

    total_adjusted = statistics.mean([traditional_econs_adjusted, behavioural_econs_adjusted])
    return(total_adjusted)

### Effect of Loss Adversion on Popularity

In [4]:
# [Between 0 and 1]
def popularity_due_to_loss_aversion(qty):
    return(math.exp(-qty/loss_aversion_coefficient))

### Effects of Saliency Bias on Popularity (TO DO)

In [5]:
def popularity_due_to_saliency_bias(size):
    return sigmoid(size)

### Effects of Ratings on Popularity

In [6]:
# [Between 0 and 1]
def popularity_due_to_rating(pdt_rating, brand_rating):
    raw_brand = (0.8*pdt_rating + 0.2*brand_rating)
    return(raw_brand/5)

## Creating Product Class and List

In [7]:
pdt_list = []
class product:
    # Popularity Coefficients
    pop_loss_adversion_coefficient = 0.33
    pop_saliency_coefficient = 0.33
    pop_rating_coefficient = 0.33
    
    
    # Raw Data
    def __init__(self, index, name, department, product_category, product_type, brand, initial_price, discounted_price, qty, customer_rating, brand_rating=5, percentage_usage=0.5, size=0):
        self.name = name
        self.index = index
        self.department = department
        self.product_category = product_category
        self.product_type = product_type
        self.brand = brand
        self.initial_price = initial_price
        self.discounted_price = discounted_price
        self.qty = qty
        self.customer_rating = customer_rating
        self.brand_rating = brand_rating
        self.percentage_usage = percentage_usage
        self.size = size
        
        self.popularity = 0
    
    # Processed Data
    
    def set_popularity(self):
        discount_factor = popularity_due_to_discount(self.initial_price, self.discounted_price, self.percentage_usage)
        loss_adversion_factor = popularity_due_to_loss_aversion(self.qty)
        saliency_factor = popularity_due_to_saliency_bias(self.size)
        rating_factor = popularity_due_to_rating(self.customer_rating, self.brand_rating)
        
        initial_popularity = self.pop_loss_adversion_coefficient*loss_adversion_factor + self.pop_saliency_coefficient*saliency_factor + self.pop_rating_coefficient*rating_factor
        self.popularity = sigmoid(initial_popularity + discount_factor)
        
        
        

In [8]:
# Populating the Product List
for index, row in pdt_csv_data.iterrows():
    cur_pdt = product(index, row["name"], row["department"], row["product_category"], row["product_type"], row["brand"], row["initial_price"], row["discounted_price"], row["qty"], row["customer_rating"])
    cur_pdt.set_popularity()
    
    pdt_list.append(cur_pdt)


In [9]:
pdt_list[4].index

4

## Creating Shelf Class and Layout Object

In [10]:
class shelf:
    # Class Variables
    max_capacity = 100
    
    def __init__(self):
        self.pdts = []
        self.pdt_set = set()
        self.cur_capacity = 0
    
    def add_pdt(self, pdt):
        # If shelf can accomodate the product
        if self.cur_capacity + pdt.size <= self.max_capacity:
            self.cur_capacity += pdt.size
            self.pdts.append(pdt)
            self.pdt_set.add(pdt.index)
            return 0
        
        # If shelf is full
        return -1
    
class layout:
    
    def __init__(self, grid, shelf_list=[], pdt_list=[]):
        # Grid is a 2d matrix where shelves are 1 indexed
        self.grid = copy.deepcopy(grid)
        self.shelf_list = copy.deepcopy(shelf_list)
        
        # A* Grid is a grid where shelves are labelled as 1
        self.a_star_grid = copy.deepcopy(grid)
        for i in range(len(self.a_star_grid)):
            for j in range(len(self.a_star_grid)):
                if self.a_star_grid[i][j] > 1:
                    self.a_star_grid[i][j] = 1
                    
        # pdt_list contains the products that exists somewhere within the layout
        self.pdt_list = copy.deepcopy(pdt_list)
        
        self.entrance = [7,7]
        
    # A_Star Performs a simulation of a person walking within the layout from init to goal
    # A_Star Returns a list of nodes visited on the path
    # Coordinates are written as [y,x] with [0,0] being the upper left hand corner
    def a_star(self, init, goal): 
        grid = copy.deepcopy(self.a_star_grid)
        
        # init = [0, 0]
        # goal = [len(grid) - 1, len(grid[0]) - 1]  # all coordinates are given in format [y,x]
        
        cost = 1

        # the cost map which pushes the path closer to the goal
        heuristic = [[0 for row in range(len(grid[0]))] for col in range(len(grid))]
        for i in range(len(grid)):
            for j in range(len(grid[0])):
                heuristic[i][j] = abs(i - goal[0]) + abs(j - goal[1])
                if grid[i][j] == 1:
                    heuristic[i][j] = 99  # added extra penalty in the heuristic map


        # the actions we can take
        delta = [[-1, 0], [0, -1], [1, 0], [0, 1]]  # go up  # go left  # go down  # go right


        # function to search the path
        def search(grid, init, goal, cost, heuristic):

            closed = [
                [0 for col in range(len(grid[0]))] for row in range(len(grid))
            ]  # the reference grid
            closed[init[0]][init[1]] = 1
            action = [
                [0 for col in range(len(grid[0]))] for row in range(len(grid))
            ]  # the action grid

            x = init[0]
            y = init[1]
            g = 0
            f = g + heuristic[init[0]][init[0]]
            cell = [[f, g, x, y]]

            found = False  # flag that is set when search is complete
            resign = False  # flag set if we can't find expand

            while not found and not resign:
                if len(cell) == 0:
                    return "FAIL"
                else:
                    cell.sort()  # to choose the least costliest action so as to move closer to the goal
                    cell.reverse()
                    next = cell.pop()
                    x = next[2]
                    y = next[3]
                    g = next[1]

                    if x == goal[0] and y == goal[1]:
                        found = True
                    else:
                        for i in range(len(delta)):  # to try out different valid actions
                            x2 = x + delta[i][0]
                            y2 = y + delta[i][1]
                            if x2 >= 0 and x2 < len(grid) and y2 >= 0 and y2 < len(grid[0]):
                                if closed[x2][y2] == 0 and grid[x2][y2] == 0:
                                    g2 = g + cost
                                    f2 = g2 + heuristic[x2][y2]
                                    cell.append([f2, g2, x2, y2])
                                    closed[x2][y2] = 1
                                    action[x2][y2] = i
            invpath = []
            x = goal[0]
            y = goal[1]
            invpath.append([x, y])  # we get the reverse path from here
            while x != init[0] or y != init[1]:
                x2 = x - delta[action[x][y]][0]
                y2 = y - delta[action[x][y]][1]
                x = x2
                y = y2
                invpath.append([x, y])

            path = []
            for i in range(len(invpath)):
                path.append(invpath[len(invpath) - 1 - i])
                
#             print("ACTION MAP")
#             for i in range(len(action)):
#                 print(action[i])

            return path
        return search(grid, init, goal, cost, heuristic)
    
    # Simulates Choice of Object to Buy and Deletes that Object from Object List
    def choose_pdt(self):
        self.pdt_list.sort(key=lambda x: x.popularity, reverse=True)
        chosen_pdt = pdt_list[0]
        pdt_list.pop()
        
        return chosen_pdt
        
    # Simulates a person entering shop
    def new_shopper(self):
        chosen_pdt = self.choose_pdt()
        print("Chosen Product", chosen_pdt.index)
        
        for shelf_index, shelf in enumerate(self.shelf_list):
            if chosen_pdt.index in shelf.pdt_set:
                #print("Found Object")
                chosen_shelf_index = shelf_index
                
                # Remove product from shelf
                shelf.pdt_set.discard(chosen_pdt.index)
                for i, pdt in enumerate(shelf.pdts):
                    if pdt.index == chosen_pdt.index:
                        del shelf.pdts[i]
                        break
                        
        print("Shelf Containing Product", chosen_shelf_index)
        
        for y in range(len(self.grid)):
            for x in range(len(self.grid[0])):
                if self.grid[y][x] == chosen_shelf_index:
                    shelf_location = (y,x)
                    
        delta = [(1,0), (-1,0), (0,1), (0,-1)]
        for d in delta:
            y = shelf_location[0]+d[0]
            x = shelf_location[1]+d[1]
            if x >= 0 and x < len(self.grid) and y >= 0 and y < len(self.grid[0]):
                
                path = self.a_star(self.entrance, [y,x])
                if path == "FAIL":
                    continue
                else:
                    return path
                    
                
        
        print("Unable to Reach Object")
        return
        

In [11]:
## Testing the Layout Object
example_layout_grid = [
    [0,0,0,0,0,0,0,0],
    [0,0,1,0,0,2,0,0],
    [0,0,3,0,0,4,0,0],
    [0,0,11,0,0,6,0,0],
    [0,0,12,0,0,7,0,0],
    [0,0,13,0,0,8,0,0],
    [0,0,14,0,0,9,0,0],
    [0,0,15,0,0,10,0,0]
]
goal = [7,1]
start = [7,7]

pdt_1 = pdt_list[0]
shelf_0 = shelf()
shelf_1 = shelf()
shelf_1.add_pdt(pdt_1)

test_layout = layout(example_layout_grid, [shelf_0, shelf_1], [pdt_1])

optimal_path = test_layout.new_shopper()
annotated_grid = copy.deepcopy(example_layout_grid)
for cell in optimal_path:
    annotated_grid[cell[0]][cell[1]] = -1

pprint(annotated_grid)

Chosen Product 0
Shelf Containing Product 1
[[0, 0, -1, -1, -1, -1, -1, -1],
 [0, 0, 1, 0, 0, 2, 0, -1],
 [0, 0, 3, 0, 0, 4, 0, -1],
 [0, 0, 11, 0, 0, 6, 0, -1],
 [0, 0, 12, 0, 0, 7, 0, -1],
 [0, 0, 13, 0, 0, 8, 0, -1],
 [0, 0, 14, 0, 0, 9, 0, -1],
 [0, 0, 15, 0, 0, 10, 0, -1]]
